In [ ]:
"""
Minimal NVT MD simulation for fcc Cu using EMT potential
(Jupyter Notebook version)
"""

import os

from ase.calculators.emt import EMT
from ase.md.langevin import Langevin
from ase import units
from ase.io import Trajectory

from learnmlp.systems.cu_fcc import CuFCCBuilder


In [ ]:
def run_nvt_md(
    temperature: float = 300.0,
    timestep_fs: float = 1.0,
    friction: float = 0.01,
    n_steps: int = 1000,
    output_dir: str = "data/raw",
    traj_interval: int = 1,
):
    """
    Run a minimal NVT MD simulation (Langevin thermostat).

    Parameters
    ----------
    temperature : float
        Temperature in Kelvin.
    timestep_fs : float
        MD timestep in femtoseconds.
    friction : float
        Langevin friction coefficient (1/fs).
    n_steps : int
        Number of MD steps.
    output_dir : str
        Directory to save trajectory.
    traj_interval : int
        Interval (in MD steps) for trajectory output.
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Build atomic system
    builder = CuFCCBuilder()
    atoms = builder.build()

    # Assign EMT calculator
    atoms.calc = EMT()

    # Set up Langevin dynamics (NVT)
    dyn = Langevin(
        atoms,
        timestep_fs * units.fs,
        temperature_K=temperature,
        friction=friction,
    )

    # Trajectory output
    traj_file = os.path.join(
        output_dir, f"cu_fcc_nvt_{int(temperature)}K.traj"
    )
    traj = Trajectory(traj_file, "w", atoms)
    dyn.attach(traj.write, interval=traj_interval)

    # Run MD
    print(f"Running NVT MD at {temperature:.0f} K ...")
    dyn.run(n_steps)

    print(f"MD finished. Trajectory saved to:\n  {traj_file}")

    return traj_file


In [ ]:
traj_path = run_nvt_md(
    temperature=300,
    timestep_fs=1.0,
    friction=0.01,
    n_steps=2000,
    output_dir="data/raw",
)


In [ ]:
temperatures = [300, 600, 900]
traj_files = []

for T in temperatures:
    traj_file = run_nvt_md(
        temperature=T,
        timestep_fs=1.0,
        friction=0.01,
        n_steps=2000,
        output_dir="data/raw",
    )
    traj_files.append(traj_file)

traj_files
